# TODO: Zklend

In [1]:
import pickle
import pandas

from classes import State, Prices
import constants
import db

## Load and prepare events

In [2]:
# Establish the connection.
connection = db.establish_connection()

# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql=f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address IN (
            '{constants.Protocol.ZKLEND.value}',
            '{constants.get_address("zETH")}',
            '{constants.get_address("zWBTC")}',
            '{constants.get_address("zUSDC")}',
            '{constants.get_address("zUSDT")}',
            '{constants.get_address("zDAI")}',
            '{constants.get_address("zETH")}'
        ) 
    AND
        key_name IN (
            'Deposit',
            'Withdrawal',
            'CollateralEnabled',
            'CollateralDisabled',
            'Borrowing',
            'Repayment',
            'Liquidation',
            'AccumulatorsSync',
            'Transfer'
        )
    ORDER BY
        block_number ASC, id;
    """,
    con=connection,
)

# Close the connection.
connection.close()

/tmp/ipykernel_244142/1668267694.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  zklend_events = pandas.read_sql(


In [3]:
zklend_events.set_index('id', inplace = True)

# Process events

Events examples:
- [Deposit](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_3)
- [AccumulatorsSync](https://starkscan.co/event/0x029628b89875a98c1c64ae206e7eb65669cb478a24449f3485f5e98aba6204dc_0)
- [CollateralEnabled](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_6)
- [CollateralDisabled](https://starkscan.co/event/0x0049b445bed84e0118795dbd22d76610ccac2ad626f8f04a1fc7e38113c2afe7_0)
- [Withdrawal](https://starkscan.co/event/0x03472cf7511687a55bc7247f8765c4bbd2c18b70e09b2a10a77c61f567bfd2cb_4)
- [Borrowing](https://starkscan.co/event/0x076b1615750528635cf0b63ca80986b185acbd20fa37f0f2b5368a4f743931f8_3)
- [Repayment](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_5)
- [Liquidation](https://starkscan.co/event/0x07b8ec709df1066d9334d56b426c45440ca1f1bb841285a5d7b33f9d1008f256_5)

In [4]:
# Iterate over ordered events to obtain the final state of each user.
state = State()
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

In [5]:
# Initialize prices
prices = Prices()

# TODO: Debugging negative deposits below

In [6]:
# TODO: negative deposits?
print("Action   Token   Raw value   Raw $   Readable $")
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min(), f"${prices.to_dollars(pandas.Series(values).min(), token)}", prices.to_dollars_pretty(pandas.Series(values).min(), token))

Action   Token   Raw value   Raw $   Readable $
deposit ETH -574696913.78003945354 $-0.000001114268352189842958401690985 $0
deposit wBTC 0 $0 $0
deposit USDC -15.6281255282999945034185 $-0.00001562757854390650389034547946 $-0.00002
deposit DAI 0 $0E-51 $0
deposit USDT -4997676.927172805096183238170 $-4.995482947001776398698594113 $-4.99548
borrowings ETH 0 $0E-40 $0
borrowings wBTC 0 $0 $0
borrowings USDC 0 $0E-51 $0
borrowings DAI 0 $0E-51 $0
borrowings USDT 0 $0E-53 $0


In [8]:
# TODO: negative zTokens
for attr in ['balance']:
    for token in ['zETH', 'zWBTC', 'zUSDC', 'zDAI', 'zUSDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(token, pandas.Series(values).min())

AttributeError: 'tuple' object has no attribute 'token_states'

In [ ]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]

[('0x4a587e33647247ed71d178b25155a27463765f7c113f05904126692d348cab1',
  Decimal('-4997676.927172805096183238170'))]

## Store state into a file

In [ ]:
filehandler = open("state.pckl", 'wb')
pickle.dump(state, filehandler)

## Load state from the file

In [ ]:
filehandler = open("state.pckl", 'rb')
state = pickle.load(filehandler)

In [ ]:
# TODO: Get token prices here: https://www.coingecko.com/en/api/documentation